In [2]:
#this is where I will be merging all 3 of my data sets
import pandas as pd
import numpy as np


In [24]:
#loading in the data sets
regular1 = pd.read_csv("Raw Data/regular1.csv")
regular2 = pd.read_csv("Raw Data/regular 2.csv")
regular3 = pd.read_csv("Raw Data/regular 3.csv")
totals = pd.read_csv("Raw Data/regulartotals.csv")


In [22]:
#merging the data sets for simplicty
stats = pd.concat([regular1, regular2, regular3], ignore_index=True)

In [34]:
#renaming the columns of the next file so they can merge
stats.rename(columns={
    'SEASON_YEAR': 'season_year',
    'TEAM_ID': 'teamId',
    'GAME_ID': 'gameId'}, inplace=True)


In [36]:
stats.head()

,season_year,game_date,gameId,matchup,teamId,teamCity,teamName,teamTricode,teamSlug,personId,...,reboundsOffensive,reboundsDefensive,reboundsTotal,assists,steals,blocks,turnovers,foulsPersonal,points,plusMinusPoints
0,2010-11,2010-11-10,21000112,NJN @ CLE,1610612751,New Jersey,Nets,NJN,nets,693,...,0,0,0,0,0,0,0,0,0,-4
1,2010-11,2010-11-17,21000165,NJN @ UTA,1610612751,New Jersey,Nets,NJN,nets,693,...,0,0,0,0,0,0,0,0,0,0
2,2010-11,2010-11-27,21000237,NJN @ PHI,1610612751,New Jersey,Nets,NJN,nets,693,...,0,0,0,0,0,0,0,0,0,0
3,2010-11,2010-12-12,21000351,NJN vs. LAL,1610612751,New Jersey,Nets,NJN,nets,693,...,0,0,0,0,0,0,0,0,0,0
4,2010-11,2010-10-29,21000020,NJN vs. SAC,1610612751,New Jersey,Nets,NJN,nets,693,...,0,1,1,1,0,0,1,1,2,6


In [37]:
totals.head()

,SEASON_YEAR,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,...,AST_RANK,TOV_RANK,STL_RANK,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK,AVAILABLE_FLAG
0,2022-23,1610612744,GSW,Golden State Warriors,22201230,2023-04-09T00:00:00,GSW @ POR,W,48.0,58,...,1,1578,55,512,457,678,2455,3,1,1.0
1,2020-21,1610612749,MIL,Milwaukee Bucks,22000051,2020-12-29T00:00:00,MIL @ MIA,W,48.0,51,...,146,1636,33,1830,1361,1552,1220,14,7,1.0
2,2013-14,1610612751,BKN,Brooklyn Nets,21300359,2013-12-16T00:00:00,BKN vs. PHI,W,48.0,47,...,22,2280,406,1979,1,1004,453,19,12,1.0
3,2013-14,1610612757,POR,Portland Trail Blazers,21300347,2013-12-14T00:00:00,POR @ PHI,W,48.0,52,...,2,2065,628,204,1269,272,2190,4,15,1.0
4,2018-19,1610612745,HOU,Houston Rockets,21801200,2019-04-07T00:00:00,HOU vs. PHX,W,48.0,53,...,84,176,149,917,1,244,1725,4,20,1.0


In [42]:
#renaming the columns of the totals file so they can merge
#one file had upper case column names and the other had lower case
totals.rename(columns={
    'SEASON_YEAR': 'season_year',
    'TEAM_ID': 'teamId',
    'GAME_ID': 'gameId'
}, inplace=True)

#using pd.merge to merge the two data sets on season year, team id, and game id
winloss = pd.merge(
    stats,
    totals[['season_year', 'teamId', 'gameId', 'WL']],
    on=['season_year', 'teamId', 'gameId'],
    how='left'
)


In [ ]:

#creating a new csv so I don't lose the data
winloss.to_csv("merged_winloss_data.csv", index=False)

In [51]:
#filtering the players in the merged data set by the players involved in the trade
players = ['Kawhi Leonard', 'DeMar DeRozan']
tradedata1 = winloss[winloss['personName'].isin(players)]

In [ ]:
#this code aggregates the data by player and season, calculating total games played, total points scored, and average plus-minus points
#the total points data will be divided by the games played to get a points per game average

tradestats1 = tradedata1.groupby(['personName', 'season_year']).agg(
    games_played=('gameId', 'nunique'),
    total_points=('points', 'sum'),
    avg_plus_minus=('plusMinusPoints', 'mean')
).reset_index()


In [61]:
# creating a new points per game column calculated by total points divided by games played
tradestats1['ppg'] = tradestats1['total_points'] / tradestats1['games_played'] 
#reordering the columns for better readability
tradestats1 = tradestats1[['personName', 'season_year', 'games_played', 'ppg', 'avg_plus_minus']]
tradestats1

,personName,season_year,games_played,ppg,avg_plus_minus
0,DeMar DeRozan,2010-11,82,17.195122,-5.012195
1,DeMar DeRozan,2011-12,66,15.969697,-1.409091
2,DeMar DeRozan,2012-13,82,18.109756,-1.365854
3,DeMar DeRozan,2013-14,81,22.111111,2.246914
4,DeMar DeRozan,2014-15,61,19.737705,2.147541
5,DeMar DeRozan,2015-16,81,22.592593,2.148148
6,DeMar DeRozan,2016-17,74,27.297297,1.986486
7,DeMar DeRozan,2017-18,82,22.439024,5.390244
8,DeMar DeRozan,2018-19,79,20.696203,0.126582
9,DeMar DeRozan,2019-20,70,21.485714,-1.685714
